In [49]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import soundfile as sf
from scipy import signal
import os

In [50]:
song = 'crooked.wav'
input_file = os.path.join('input', song)
ir_file = 'impulse_responses/EchoThiefImpulseResponseLibrary/EchoThiefImpulseResponseLibrary/Venues/SteinmanHall.wav'
output_file = os.path.join('output', song)

In [51]:
def read_input(wavfile):
    """
    Reads in the wavfile as a numpy array and cleans and normalizes the signal. 
    Returns the normalized signal and the sampling frequency.
    
    Args:
        wavfile: file path to .wav file
    
    Returns:
        signal: numpy array of the wavfile sampled at fs
        fs: sampling frequency
    """
    signal, fs = sf.read(wavfile)
    signal = signal.astype(np.float64)    # Cleaning
    signal = signal / np.abs(np.max(signal))    # Normalization
    
    return signal, fs

def fft_convolve(x, h):
    """
    Convolved the signal x with an impulse response h. Returns the convolved signal trimmed to the same length as the input.
    
    Args:
        x: input signal
        h: impulse response
    
    Returns:
        input signal convolved with the impulse response, trimmed to be the same length as the input signal
    """
    result = signal.fftconvolve(x, h, mode='full')
    
    # Find where the impulse response is strongest; this is the "start" of the impulse response
    # Accounts for impulse responses with a long pause before the actual response
    p_max = np.argmax(np.abs(h))
    
    # Compensate for impulse response delay by shifting the convolved signal
    output = np.empty_like(result)
    shift = -p_max
    if shift >= 0:
        output[:shift] = 0.0
        output[shift:] = result[:shift]
    else:
        output[shift:] = 0.0
        output[:shift] = result[-shift:]
    
    # Trim the convolved signal to be the same length as the original
    return output[0: x.shape[0]]

In [52]:
sgnl, fs = read_input(input_file)
ir, fs_ir = read_input(ir_file)

In [53]:
signal_rev = fft_convolve(sgnl, ir)

In [54]:
sf.write(output_file, signal_rev, fs)